# Modelling

Where in the future machine learning models(Neural Nets) will be implemented to improve the recommender's accuracy.

In [1]:
import polars as pl
import sys 
import os 

# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from modelling import *

In [4]:
# importing data 
df = pl.read_csv("../data/google_analytics_data.csv",ignore_errors=True)
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
i64,i64,i64,str,str,str,str,str,str,str,str,str,str
20170731,1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
20170731,1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
20170731,1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
20170731,1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
20170731,1501520683,2219384770970157334,"""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [ ]:
# Finds average accuracy across all users 
# purchasing_visitors=encoded_df.filter(pl.col('transactionId') != 0).select(pl.col('fullVisitorId')).unique()
# all_recs = []
# for i in purchasing_visitors["fullVisitorId"].to_list():
#     prec = recommendation_eval_pipeline(i, user_profiles, encoded_df, entire_data)
#     all_recs.append(prec)
# np.mean(all_recs)